TPU版ACNET：
实现未在原数据集，这里使用的是TPU中一个104种花朵分类的数据集
stage1：acc=89%左右
未曾仔细调参，只是证明代码可收敛并在TPU上可使用
可直接执行全部代码，前提是需在kaggle下的jupyter才能使用数据集


 细粒度分类
 
 reference：
 
 https://isrc.iscas.ac.cn/gitlab/research/acnet
 
 https://arxiv.org/pdf/1909.11378.pdf
 
 tensorflow==2.3.0
 
 
 基本完全按照原文的思路实现的树形结构，但在参数设置、各个模块的实现细节等方面有参照以下
 
 https://github.com/FlyingMoon-GitHub/ACNet

In [81]:
import math, re, os
import tensorflow as tf
import numpy as np
from matplotlib import pyplot as plt
from kaggle_datasets import KaggleDatasets
from sklearn.metrics import f1_score, precision_score, recall_score, confusion_matrix
print("Tensorflow version " + tf.__version__)
AUTO = tf.data.experimental.AUTOTUNE

Tensorflow version 2.4.1


# TPU or GPU detection

In [82]:
# NEW on TPU in TensorFlow 24: shorter cross-compatible TPU/GPU/multi-GPU/cluster-GPU detection code

try: # detect TPUs
    tpu = tf.distribute.cluster_resolver.TPUClusterResolver.connect() # TPU detection
    strategy = tf.distribute.TPUStrategy(tpu)
except ValueError: # detect GPUs
    strategy = tf.distribute.MirroredStrategy() # for GPU or multi-GPU machines
    #strategy = tf.distribute.get_strategy() # default strategy that works on CPU and single GPU
    #strategy = tf.distribute.experimental.MultiWorkerMirroredStrategy() # for clusters of multi-GPU machines

print("Number of accelerators: ", strategy.num_replicas_in_sync)

Number of accelerators:  8


# Competition data access
TPUs read data directly from Google Cloud Storage (GCS). This Kaggle utility will copy the dataset to a GCS bucket co-located with the TPU. If you have multiple datasets attached to the notebook, you can pass the name of a specific dataset to the get_gcs_path function. The name of the dataset is the name of the directory it is mounted in. Use `!ls /kaggle/input/` to list attached datasets.

In [ ]:
GCS_DS_PATH = KaggleDatasets().get_gcs_path() # you can list the bucket with "!gsutil ls $GCS_DS_PATH"

# Configuration

In [ ]:
IMAGE_SIZE = [512, 512] # At this size, a GPU will run out of memory. Use the TPU.
                        # For GPU training, please select 224 x 224 px image size.
EPOCHS = 12
BATCH_SIZE = 16 * strategy.num_replicas_in_sync
PER_REPLICA_BATCH_SIZE = 16

GCS_PATH_SELECT = { # available image sizes
    192: GCS_DS_PATH + '/tfrecords-jpeg-192x192',
    224: GCS_DS_PATH + '/tfrecords-jpeg-224x224',
    331: GCS_DS_PATH + '/tfrecords-jpeg-331x331',
    512: GCS_DS_PATH + '/tfrecords-jpeg-512x512'
}
GCS_PATH = GCS_PATH_SELECT[IMAGE_SIZE[0]]

TRAINING_FILENAMES = tf.io.gfile.glob(GCS_PATH + '/train/*.tfrec')
VALIDATION_FILENAMES = tf.io.gfile.glob(GCS_PATH + '/val/*.tfrec')
TEST_FILENAMES = tf.io.gfile.glob(GCS_PATH + '/test/*.tfrec') # predictions on this dataset should be submitted for the competition

CLASSES = ['pink primrose',    'hard-leaved pocket orchid', 'canterbury bells', 'sweet pea',     'wild geranium',     'tiger lily',           'moon orchid',              'bird of paradise', 'monkshood',        'globe thistle',         # 00 - 09
           'snapdragon',       "colt's foot",               'king protea',      'spear thistle', 'yellow iris',       'globe-flower',         'purple coneflower',        'peruvian lily',    'balloon flower',   'giant white arum lily', # 10 - 19
           'fire lily',        'pincushion flower',         'fritillary',       'red ginger',    'grape hyacinth',    'corn poppy',           'prince of wales feathers', 'stemless gentian', 'artichoke',        'sweet william',         # 20 - 29
           'carnation',        'garden phlox',              'love in the mist', 'cosmos',        'alpine sea holly',  'ruby-lipped cattleya', 'cape flower',              'great masterwort', 'siam tulip',       'lenten rose',           # 30 - 39
           'barberton daisy',  'daffodil',                  'sword lily',       'poinsettia',    'bolero deep blue',  'wallflower',           'marigold',                 'buttercup',        'daisy',            'common dandelion',      # 40 - 49
           'petunia',          'wild pansy',                'primula',          'sunflower',     'lilac hibiscus',    'bishop of llandaff',   'gaura',                    'geranium',         'orange dahlia',    'pink-yellow dahlia',    # 50 - 59
           'cautleya spicata', 'japanese anemone',          'black-eyed susan', 'silverbush',    'californian poppy', 'osteospermum',         'spring crocus',            'iris',             'windflower',       'tree poppy',            # 60 - 69
           'gazania',          'azalea',                    'water lily',       'rose',          'thorn apple',       'morning glory',        'passion flower',           'lotus',            'toad lily',        'anthurium',             # 70 - 79
           'frangipani',       'clematis',                  'hibiscus',         'columbine',     'desert-rose',       'tree mallow',          'magnolia',                 'cyclamen ',        'watercress',       'canna lily',            # 80 - 89
           'hippeastrum ',     'bee balm',                  'pink quill',       'foxglove',      'bougainvillea',     'camellia',             'mallow',                   'mexican petunia',  'bromelia',         'blanket flower',        # 90 - 99
           'trumpet creeper',  'blackberry lily',           'common tulip',     'wild rose']                                                                                                                                               # 100 - 102

## Visualization utilities
data -> pixels, nothing of much interest for the machine learning practitioner in this section.

In [ ]:
# numpy and matplotlib defaults
np.set_printoptions(threshold=15, linewidth=80)

def batch_to_numpy_images_and_labels(data):
    images, labels = data
    numpy_images = images.numpy()
    numpy_labels = labels.numpy()
    if numpy_labels.dtype == object: # binary string in this case, these are image ID strings
        numpy_labels = [None for _ in enumerate(numpy_images)]
    # If no labels, only image IDs, return None for labels (this is the case for test data)
    return numpy_images, numpy_labels

def title_from_label_and_target(label, correct_label):
    if correct_label is None:
        return CLASSES[label], True
    correct = (label == correct_label)
    return "{} [{}{}{}]".format(CLASSES[label], 'OK' if correct else 'NO', u"\u2192" if not correct else '',
                                CLASSES[correct_label] if not correct else ''), correct

def display_one_flower(image, title, subplot, red=False, titlesize=16):
    plt.subplot(*subplot)
    plt.axis('off')
    plt.imshow(image)
    if len(title) > 0:
        plt.title(title, fontsize=int(titlesize) if not red else int(titlesize/1.2), color='red' if red else 'black', fontdict={'verticalalignment':'center'}, pad=int(titlesize/1.5))
    return (subplot[0], subplot[1], subplot[2]+1)
    
def display_batch_of_images(databatch, predictions=None):
    """This will work with:
    display_batch_of_images(images)
    display_batch_of_images(images, predictions)
    display_batch_of_images((images, labels))
    display_batch_of_images((images, labels), predictions)
    """
    # data
    images, labels = batch_to_numpy_images_and_labels(databatch)
    if labels is None:
        labels = [None for _ in enumerate(images)]
        
    # auto-squaring: this will drop data that does not fit into square or square-ish rectangle
    rows = int(math.sqrt(len(images)))
    cols = len(images)//rows
        
    # size and spacing
    FIGSIZE = 13.0
    SPACING = 0.1
    subplot=(rows,cols,1)
    if rows < cols:
        plt.figure(figsize=(FIGSIZE,FIGSIZE/cols*rows))
    else:
        plt.figure(figsize=(FIGSIZE/rows*cols,FIGSIZE))
    
    # display
    for i, (image, label) in enumerate(zip(images[:rows*cols], labels[:rows*cols])):
        title = '' if label is None else CLASSES[label]
        correct = True
        if predictions is not None:
            title, correct = title_from_label_and_target(predictions[i], label)
        dynamic_titlesize = FIGSIZE*SPACING/max(rows,cols)*40+3 # magic formula tested to work from 1x1 to 10x10 images
        subplot = display_one_flower(image, title, subplot, not correct, titlesize=dynamic_titlesize)
    
    #layout
    plt.tight_layout()
    if label is None and predictions is None:
        plt.subplots_adjust(wspace=0, hspace=0)
    else:
        plt.subplots_adjust(wspace=SPACING, hspace=SPACING)
    plt.show()

def display_confusion_matrix(cmat, score, precision, recall):
    plt.figure(figsize=(15,15))
    ax = plt.gca()
    ax.matshow(cmat, cmap='Reds')
    ax.set_xticks(range(len(CLASSES)))
    ax.set_xticklabels(CLASSES, fontdict={'fontsize': 7})
    plt.setp(ax.get_xticklabels(), rotation=45, ha="left", rotation_mode="anchor")
    ax.set_yticks(range(len(CLASSES)))
    ax.set_yticklabels(CLASSES, fontdict={'fontsize': 7})
    plt.setp(ax.get_yticklabels(), rotation=45, ha="right", rotation_mode="anchor")
    titlestring = ""
    if score is not None:
        titlestring += 'f1 = {:.3f} '.format(score)
    if precision is not None:
        titlestring += '\nprecision = {:.3f} '.format(precision)
    if recall is not None:
        titlestring += '\nrecall = {:.3f} '.format(recall)
    if len(titlestring) > 0:
        ax.text(101, 1, titlestring, fontdict={'fontsize': 18, 'horizontalalignment':'right', 'verticalalignment':'top', 'color':'#804040'})
    plt.show()
    
def display_training_curves(training, validation, title, subplot):
    if subplot%10==1: # set up the subplots on the first call
        plt.subplots(figsize=(10,10), facecolor='#F0F0F0')
        plt.tight_layout()
    ax = plt.subplot(subplot)
    ax.set_facecolor('#F8F8F8')
    ax.plot(training)
    ax.plot(validation)
    ax.set_title('model '+ title)
    ax.set_ylabel(title)
    #ax.set_ylim(0.28,1.05)
    ax.set_xlabel('epoch')
    ax.legend(['train', 'valid.'])

# Datasets

In [ ]:
def decode_image(image_data):
    image = tf.image.decode_jpeg(image_data, channels=3)  # image format uint8 [0,255]
    image = tf.reshape(image, [*IMAGE_SIZE, 3]) # explicit size needed for TPU
    return image

def read_labeled_tfrecord(example):
    LABELED_TFREC_FORMAT = {
        "image": tf.io.FixedLenFeature([], tf.string), # tf.string means bytestring
        "class": tf.io.FixedLenFeature([], tf.int64),  # shape [] means single element
    }
    example = tf.io.parse_single_example(example, LABELED_TFREC_FORMAT)
    image = decode_image(example['image'])
    label = tf.cast(example['class'], tf.int32)
    return image, label # returns a dataset of (image, label) pairs

def read_unlabeled_tfrecord(example):
    UNLABELED_TFREC_FORMAT = {
        "image": tf.io.FixedLenFeature([], tf.string), # tf.string means bytestring
        "id": tf.io.FixedLenFeature([], tf.string),  # shape [] means single element
        # class is missing, this competitions's challenge is to predict flower classes for the test dataset
    }
    example = tf.io.parse_single_example(example, UNLABELED_TFREC_FORMAT)
    image = decode_image(example['image'])
    idnum = example['id']
    return image, idnum # returns a dataset of image(s)

def load_dataset(filenames, labeled=True, ordered=False):
    # Read from TFRecords. For optimal performance, reading from multiple files at once and
    # disregarding data order. Order does not matter since we will be shuffling the data anyway.

    ignore_order = tf.data.Options()
    if not ordered:
        ignore_order.experimental_deterministic = False # disable order, increase speed

    dataset = tf.data.TFRecordDataset(filenames, num_parallel_reads=AUTO) # automatically interleaves reads from multiple files
    dataset = dataset.with_options(ignore_order) # uses data as soon as it streams in, rather than in its original order
    dataset = dataset.map(read_labeled_tfrecord if labeled else read_unlabeled_tfrecord, num_parallel_calls=AUTO)
    # returns a dataset of (image, label) pairs if labeled=True or (image, id) pairs if labeled=False
    return dataset

def data_augment(image, label):
    # data augmentation. Thanks to the dataset.prefetch(AUTO) statement in the next function (below),
    # this happens essentially for free on TPU. Data pipeline code is executed on the "CPU" part
    # of the TPU while the TPU itself is computing gradients.
    image = tf.image.random_flip_left_right(image)
    image = tf.image.random_hue(image, max_delta=0.2)
    image =  tf.image.stateless_random_contrast(image, lower=0.1, upper=0.9, seed=(2, 0))
    image = tf.image.random_crop(image, (448, 448, 3))
    image = tf.image.random_saturation(image, 0, 2)
    return image, label   

def test_resize(image, label):
    # data augmentation. Thanks to the dataset.prefetch(AUTO) statement in the next function (below),
    # this happens essentially for free on TPU. Data pipeline code is executed on the "CPU" part
    # of the TPU while the TPU itself is computing gradients.
    image = tf.image.resize(image, (448, 448))
    #image = tf.image.random_saturation(image, 0, 2)
    return image, label

def get_training_dataset():
    dataset = load_dataset(TRAINING_FILENAMES, labeled=True)
    dataset = dataset.map(data_augment, num_parallel_calls=AUTO)
    dataset = dataset.repeat() # the training dataset must repeat for several epochs
    dataset = dataset.shuffle(2048)
    dataset = dataset.batch(PER_REPLICA_BATCH_SIZE)
    dataset = dataset.prefetch(AUTO) # prefetch next batch while training (autotune prefetch buffer size)
    return dataset

def get_validation_dataset(ordered=False):
    dataset = load_dataset(VALIDATION_FILENAMES, labeled=True,)
    dataset = dataset.map(test_resize, num_parallel_calls=AUTO)
    dataset = dataset.repeat() # the training dataset must repeat for several epochs
    dataset = dataset.shuffle(2048)
    dataset = dataset.batch(PER_REPLICA_BATCH_SIZE)
    dataset = dataset.prefetch(AUTO) # prefetch next batch while training (autotune prefetch buffer size)
    return dataset

def get_test_dataset(ordered=False):
    dataset = load_dataset(TEST_FILENAMES, labeled=False, ordered=ordered)
    dataset = dataset.batch(PER_REPLICA_BATCH_SIZE)
    dataset = dataset.prefetch(AUTO) # prefetch next batch while training (autotune prefetch buffer size)
    return dataset

def count_data_items(filenames):
    # the number of data items is written in the name of the .tfrec files, i.e. flowers00-230.tfrec = 230 data items
    n = [int(re.compile(r"-([0-9]*)\.").search(filename).group(1)) for filename in filenames]
    return np.sum(n)

# Dataset visualizations

In [ ]:
# data dump
print("Training data shapes:")
for image, label in get_training_dataset().take(3):
    print(image.numpy().shape, label.numpy().shape)
print("Training data label examples:", label.numpy())
print("Validation data shapes:")
for image, label in get_validation_dataset().take(3):
    print(image.numpy().shape, label.numpy().shape)
print("Validation data label examples:", label.numpy())
print("Test data shapes:")
for image, idnum in get_test_dataset().take(3):
    print(image.numpy().shape, idnum.numpy().shape)
print("Test data IDs:", idnum.numpy().astype('U')) # U=unicode string

In [ ]:
# # Peek at training data
# training_dataset = get_training_dataset()
# training_dataset = training_dataset.unbatch().batch(20)
# train_batch = iter(training_dataset)

In [ ]:
# run this cell again for next set of images
# display_batch_of_images(next(train_batch))

In [ ]:
# peer at test data
# test_dataset = get_test_dataset()
# test_dataset = test_dataset.unbatch().batch(20)
# test_batch = iter(test_dataset)

In [ ]:
# run this cell again for next set of images
# display_batch_of_images(next(test_batch))

# Model
Not the best but it converges ...

In [75]:
"""config"""
CONV_KER_INIT = 'random_normal'
CONV_BIAS_INIT = 'zeros'
DENSE_KER_INIT = 'glorot_normal'
DENSE_BIAS_INIT = 'zeros'
BASE_FTS_SIZE = 28
CLASSES_NUM = 104

In [76]:
"""Route"""
import tensorflow as tf
import tensorflow.keras as keras
import tensorflow.keras.regularizers as regularizers

class GlobalContext(keras.layers.Layer):
    def __init__(self, inplanes, planes, size, channels, pool=None, fusion=None):
        super(GlobalContext, self).__init__()
        if pool is None:
            pool = 'avg'
        if fusion is None:
            fusion = ['channel_add', 'channel_mul']
        assert pool in ['avg', 'att']
        assert all([f in ['channel_add', 'channel_mul'] for f in fusion])
        assert len(fusion) > 0, 'fusion should be used'
        self.inplanes = inplanes
        self.planes = planes
        self.pool = pool
        self.fusion = fusion
        self.size = size
        self.channels = channels

        if 'att' in pool:
            self.conv_mask = keras.layers.Conv2D(
                filters=1,
                kernel_size=1,
                strides=(1, 1),
                kernel_initializer=CONV_KER_INIT,
                bias_initializer=CONV_BIAS_INIT,
                activation='relu',
                kernel_regularizer=regularizers.l2(0.01)
            )
            self.softmax = keras.layers.Softmax(axis=-1)
        else:
            self.avg_pool = keras.layers.GlobalAveragePooling2D()
        
        if 'channel_add' in self.fusion:
            self.channel_add_conv = keras.Sequential(
                [
                 keras.layers.LayerNormalization(axis=[1, 2, 3], epsilon=1e-5),
                 keras.layers.ReLU(),
                 keras.layers.Conv2D(
                     filters=self.inplanes,
                     kernel_size=1,
                     strides=(1, 1),
                     kernel_initializer=CONV_KER_INIT,
                     bias_initializer=CONV_BIAS_INIT,
                     activation='relu',
                     kernel_regularizer=regularizers.l2(0.01)
                 )
                ]
            )
        else:
            self.channel_add_conv = None
        if 'channel_mul' in self.fusion:
            self.channel_mul_conv = keras.Sequential(
                [
                 keras.layers.Conv2D(
                     filters=self.planes,
                     kernel_size=1,
                     kernel_initializer=CONV_KER_INIT,
                     bias_initializer=CONV_BIAS_INIT,
                     activation='relu',
                     kernel_regularizer=regularizers.l2(0.01)
                 ),
                 keras.layers.LayerNormalization(axis=[1, 2, 3], epsilon=1e-5),
                 keras.layers.ReLU(),
                 keras.layers.Conv2D(
                     filters=self.inplanes,
                     kernel_size=1,
                     kernel_initializer=CONV_KER_INIT,
                     bias_initializer=CONV_BIAS_INIT,
                     activation='relu',
                     kernel_regularizer=regularizers.l2(0.01)
                 )
                ]
            )
        else:
            self.channel_mul_conv = None

    def spatial_pool(self, x):
        if self.pool == 'att':
            input_x = x
            input_x = tf.reshape(input_x, (-1, self.size*self.size, self.channels))
            input_x = tf.transpose(input_x, [0, 2, 1])
            input_x = tf.expand_dims(input_x, axis=1)

            context_mask = self.conv_mask(x)
            context_mask = tf.reshape(context_mask, (-1, self.size*self.size, 1))
            context_mask = tf.transpose(context_mask, [0, 2, 1])
            context_mask = self.softmax(context_mask)
            context_mask = context_mask.unsequeeze(3)
            context = torch.matmul(input_x, context_mask)
            context = tf.reshape(context, [-1, self.channel, 1, 1])
            context = tf.transpose(context, [0, 2, 3, 1])
        else:
            context = self.avg_pool(x)
            context = tf.reshape(context, (-1, 1, 1, self.channels))

        return context
    
    def call(self, inputs):
        context = self.spatial_pool(inputs)

        if self.channel_mul_conv is not None:
            channel_mul_term = keras.activations.sigmoid(self.channel_mul_conv(context))
            out = inputs * channel_mul_term
        else:
            out = inputs
        if self.channel_add_conv is not None:
            channel_add_term = self.channel_add_conv(context)
            out = out + channel_add_term
        return out


class Route(keras.layers.Layer):
    def __init__(self, in_channels, size, epslion=1e-12):
        super(Route, self).__init__()
        self.conv1 = keras.layers.Conv2D(
            filters = in_channels,
            kernel_size = 1,
            kernel_initializer = CONV_KER_INIT,
            bias_initializer = CONV_BIAS_INIT,
            kernel_regularizer=regularizers.l2(0.01),
            activation='relu',
        )
        self.gcblock = GlobalContext(
            inplanes = in_channels,
            planes = in_channels,
            size = size,
            channels = in_channels
        )
        self.l2norm = tf.math.l2_normalize
        self.fc = keras.layers.Dense(
            units=1,
            kernel_initializer=DENSE_KER_INIT, 
            bias_initializer=DENSE_BIAS_INIT,
            kernel_regularizer=regularizers.l2(0.01)
        )

        self.avgpool = keras.layers.GlobalAveragePooling2D()
        self.sigmoid = keras.activations.sigmoid
        self.epslion = epslion

    def call(self, inputs):
        feature = self.conv1(inputs)
        feature = self.gcblock(feature)
        feature = self.avgpool(feature)
        feature = tf.sign(feature)*tf.math.sqrt(tf.sign(feature)*feature+self.epslion)
        feature = self.l2norm(feature, axis=-1)

        out = self.fc(feature)
        out = self.sigmoid(out)

        return out

In [77]:
"""attention"""
class ASPP(keras.layers.Layer):
    def __init__(self, in_channels, depth, size):
        super(ASPP, self).__init__()
        self.size = size

        self.mean = keras.layers.GlobalAveragePooling2D()
        self.conv = keras.layers.Conv2D(
            filters=depth,
            kernel_size=1,
            kernel_initializer=CONV_KER_INIT,
            bias_initializer=CONV_BIAS_INIT,
#             activation='relu',
#             kernel_regularizer=regularizers.l2(0.01)
        )
        self.upsample = keras.layers.UpSampling2D(size, interpolation='bilinear')
        self.atrous_block1 = keras.layers.Conv2D(
            depth, 
            kernel_size=1, 
            strides=1, 
            kernel_initializer=CONV_KER_INIT,
            bias_initializer=CONV_BIAS_INIT, 
            padding='same',
#             activation='relu',
#             kernel_regularizer=regularizers.l2(0.01)
        )
        self.atrous_block6 = keras.layers.Conv2D(
            depth,
            kernel_size=3,
            strides=1,
            kernel_initializer=CONV_KER_INIT,
            bias_initializer=CONV_BIAS_INIT, 
            padding='same',
            dilation_rate=6,
#             activation='relu',
#             kernel_regularizer=regularizers.l2(0.01)
        )
        self.atrous_block12 = keras.layers.Conv2D(
            depth, 
            kernel_size=3,
            strides=1, 
            kernel_initializer=CONV_KER_INIT, 
            bias_initializer=CONV_BIAS_INIT, 
            padding='same', 
            dilation_rate=12, 
#             activation='relu',
#             kernel_regularizer=regularizers.l2(0.01)
        )
        self.atrous_block18 = keras.layers.Conv2D(
            depth, 
            kernel_size=3, 
            strides=1,
            kernel_initializer=CONV_KER_INIT,
            bias_initializer=CONV_BIAS_INIT,
            padding='same', 
            dilation_rate=18, 
#             activation='relu',
#             kernel_regularizer=regularizers.l2(0.01)
        )

        self.conv_1_output = keras.layers.Conv2D(
            depth,
            kernel_size=1,
            kernel_initializer=CONV_KER_INIT,
            bias_initializer=CONV_BIAS_INIT,
#             activation='relu',
#             kernel_regularizer=regularizers.l2(0.01)
        )
    
    def call(self, inputs):
        img_features = self.mean(inputs)
        img_features = tf.expand_dims(img_features, axis=1)
        img_features = tf.expand_dims(img_features, axis=1)
        img_features = self.conv(img_features)
        img_features = self.upsample(img_features)

        atrous_block1 = self.atrous_block1(inputs)
        atrous_block6 = self.atrous_block6(inputs)
        atrous_block12 = self.atrous_block12(inputs)
        atrous_block18 = self.atrous_block18(inputs)

        all_ = tf.concat([img_features, atrous_block1, atrous_block6, atrous_block12, atrous_block18], axis=-1)
        net = self.conv_1_output(all_)
        return net

class SEBlock(keras.layers.Layer):
    def __init__(self, in_planes, planes, stride=1):
        super(SEBlock, self).__init__()
        self.conv1 = keras.layers.Conv2D(
            filters=planes,
            kernel_size=3,
            strides=stride,
            padding='same',
            use_bias=False,
            kernel_initializer=CONV_KER_INIT,
            bias_initializer=CONV_BIAS_INIT,
            activation='relu',
            kernel_regularizer=regularizers.l2(0.01)
        )
        self.bn1 = keras.layers.BatchNormalization(momentum=0.9, epsilon=1e-5)
        self.conv2 = keras.layers.Conv2D(
            filters=planes,
            kernel_size=3,
            strides=stride,
            padding='same',
            use_bias=False,
            kernel_initializer=CONV_KER_INIT,
            bias_initializer=CONV_BIAS_INIT,
            activation='relu',
            kernel_regularizer=regularizers.l2(0.01)
        )
        self.bn2 = keras.layers.BatchNormalization(momentum=0.9, epsilon=1e-5)
        self.avgpool = keras.layers.GlobalAveragePooling2D()
        self.sigmoid = keras.activations.sigmoid

        self.shortcut = keras.Sequential([])
        if stride != 1 or in_planes != planes:
            self.shortcut = keras.Sequential(
                [
                 keras.layers.Conv2D(
                     filters=planes,
                     kernel_size=1,
                     stride=stride,
                     use_bias=False,
                     kernel_initializer=CONV_KER_INIT,
                     bias_initializer=CONV_BIAS_INIT
                 ),
                 keras.layers.BatchNormalization(momentum=0.9, epsilon=1e-5)
                ]
            )
        
        self.fc1 = keras.layers.Conv2D(
            filters=planes//16,
            kernel_size=1,
            kernel_initializer=CONV_KER_INIT,
            bias_initializer=CONV_BIAS_INIT,
            activation='relu',
            kernel_regularizer=regularizers.l2(0.01)
        )
        self.fc2 = keras.layers.Conv2D(
            filters=planes,
            kernel_size=1,
            kernel_initializer=CONV_KER_INIT,
            bias_initializer=CONV_BIAS_INIT,
            activation='relu',
            kernel_regularizer=regularizers.l2(0.01)
        )

    def call(self, inputs, training=None):
        out = keras.activations.relu(self.bn1(self.conv1(inputs)))
        out = self.bn2(self.conv2(out))

        w = self.avgpool(out)
        w = tf.expand_dims(w, axis=1)
        w = tf.expand_dims(w, axis=1)
        w = keras.activations.relu(self.fc1(w))
        w = self.sigmoid(self.fc2(w))

        out = out*w
        out += self.shortcut(inputs)
        out = keras.activations.relu(out)
        return out

class Attention(keras.layers.Layer):
    def __init__(self, in_channels):
        super(Attention, self).__init__()
        self.aspp = keras.Sequential()
#         self.aspp = ASPP(in_channels=in_channels, depth=in_channels, size=BASE_FTS_SIZE)
        self.se = SEBlock(in_planes=in_channels, planes=in_channels)
    
    def call(self, inputs):
        feature = self.aspp(inputs)
        out = self.se(feature)
        return out

In [94]:
"""LabelPred"""
class LabelPred(keras.layers.Layer):
    def __init__(self, class_num, in_channels):
        super(LabelPred, self).__init__()
        self.class_num = class_num

        self.bn = keras.layers.BatchNormalization(momentum=0.9, epsilon=1e-5)
        self.conv1 = keras.layers.Conv2D(
            filters=in_channels,
            kernel_size=1,
            kernel_initializer=CONV_KER_INIT,
            bias_initializer=CONV_BIAS_INIT,
            activation='relu',
            kernel_regularizer=regularizers.l2(0.01)
        )
        self.l2norm = tf.math.l2_normalize
        self.dropout = keras.layers.Dropout(0.3)
        self.fc = keras.layers.Dense(
            units=self.class_num,
            activation='softmax',
            kernel_initializer=DENSE_KER_INIT,
            bias_initializer=DENSE_BIAS_INIT,
            kernel_regularizer=regularizers.l2(0.01)
        )

        self.avgpool = keras.layers.GlobalAveragePooling2D()
    
    def call(self, inputs, training=None):
        feature = self.bn(inputs)
        feature = self.conv1(feature)
        feature = self.avgpool(feature)
        feature = tf.sign(feature) * tf.math.sqrt(tf.sign(feature) * feature + 1e-12)
        feature = self.l2norm(feature, axis=-1)
        
        feature =  self.dropout(feature)
        feature = self.fc(feature)
        return feature

In [95]:
"""Tree"""
class BTree(keras.layers.Layer):
    def __init__(self, in_channel, classes, size):
        super(BTree, self).__init__()
        self.route1_L1 = Route(in_channel, size)
        self.route1_L2 = Route(in_channel, size)
        self.route2_L2 = Route(in_channel, size)

        self.att_1 = Attention(in_channel)
        self.att_2 = Attention(in_channel)
        self.att_3 = Attention(in_channel)

        self.att_4 = Attention(in_channel)
        self.att_6 = Attention(in_channel)
        self.att_8 = Attention(in_channel)

        self.att_5 = Attention(in_channel)
        self.att_7 = Attention(in_channel)
        self.att_9 = Attention(in_channel)

        self.p_1 = LabelPred(classes, in_channel)
        self.p_2 = LabelPred(classes, in_channel)
        self.p_3 = LabelPred(classes, in_channel)
        self.p_4 = LabelPred(classes, in_channel)

    def call(self, inputs):
        """为了清楚搭建树结构，未使用任何循环简化代码，本树结构完全按照论文figure2实现"""
        route_result = [[0, 0], [0, 0, 0, 0]]
        features_result = [[None], [None, None], [None, None, None, None]]

        # 第一层Route
        left_prob1_l1 = self.route1_L1(inputs)
        right_prob1_l1 = 1 - left_prob1_l1
        route_result[0][0], route_result[0][1] = left_prob1_l1, right_prob1_l1

        features_result[0][0] = inputs
        # 第一层attention
        left_fts1_l1 = self.att_3(self.att_1(inputs))
        right_fts1_l1 = self.att_2(inputs)
        features_result[1][0], features_result[1][1] = left_fts1_l1, right_fts1_l1

        # 第二层Route
        left_prob1_l2 = self.route1_L2(features_result[1][0])
        right_prob1_l2 = 1 - left_prob1_l2
        left_prob2_l2 = self.route2_L2(features_result[1][1])
        right_prob2_l2 = 1 - left_prob2_l2
        route_result[1][0], route_result[1][1] = left_prob1_l2 * route_result[0][0], right_prob1_l2 * route_result[0][0]
        route_result[1][2], route_result[1][3] = left_prob2_l2 * route_result[0][1], right_prob2_l2 * route_result[0][1]

        # 第二层attention
        left_fts1_l2 = self.att_8(self.att_4(features_result[1][0]))
        right_fts1_l2 = self.att_6(features_result[1][0])
        features_result[2][0], features_result[2][1] = left_fts1_l2, right_fts1_l2

        left_fts2_l2 = self.att_9(self.att_5(features_result[1][1]))
        right_fts2_l2 = self.att_7(features_result[1][1])
        features_result[2][2], features_result[2][3] = left_fts2_l2, right_fts2_l2

        # label pred
        labelPred1 = self.p_1(features_result[2][0])
        AvgLabel = (labelPred1 * route_result[1][0])

        labelPred2 = self.p_2(features_result[2][1])
        AvgLabel += (labelPred2 * route_result[1][1])

        labelPred3 = self.p_3(features_result[2][2])
        AvgLabel += (labelPred3 * route_result[1][2])

        labelPred4 = self.p_4(features_result[2][3])
        AvgLabel += (labelPred4 * route_result[1][3])

        return labelPred1, labelPred2, labelPred3, labelPred4, AvgLabel
#         return labelPred1, labelPred2, AvgLabel


In [96]:
"""acnet"""
def ACnet(input_shape, in_channel, classes, size, firstStage=True):
    backbone = keras.applications.ResNet50(
        include_top=False,
        weights='imagenet',
        input_shape=input_shape,
    )
    if firstStage:
        backbone.trainable = False
    tree = BTree(in_channel=in_channel, classes=classes, size=size)

    my_input = backbone.layers[0].input
    output = backbone.get_layer('conv4_block6_out').output
    output = keras.layers.Conv2D(
        filters = in_channel,
        kernel_size = 1,
        kernel_initializer=CONV_KER_INIT,
        activation='relu',
        kernel_regularizer=regularizers.l2(0.01)
    )(output)
    output = tree(output)
    all_model = keras.Model(inputs=my_input, outputs=output)
    return all_model

In [83]:
model = ACnet(input_shape=(448, 448, 3), in_channel=512, classes=104, size=28,firstStage=True)
img = tf.random.normal((2, 448, 448, 3))
y = model(img)
print(y[0].shape)

(2, 104)


# loss

In [84]:
with strategy.scope():
    class myLoss(keras.losses.Loss):
        def __init__(self, alpha=0.25, betha=1.):
            super(myLoss, self).__init__(reduction=tf.keras.losses.Reduction.NONE)
            # self.loss = SparseCategoricalCrossentropy(from_logits=False)
            self.alpha = alpha
            self.betha = betha

        def call(self, y_true, y_pred):
            a = self.NLLLoss(y_true, y_pred[0]) * self.alpha
            b = self.NLLLoss(y_true, y_pred[1]) * self.alpha
            c = self.NLLLoss(y_true, y_pred[2]) * self.alpha
            d = self.NLLLoss(y_true, y_pred[3]) * self.alpha
            e = self.NLLLoss(y_true, y_pred[4]) * self.betha
            return a+b+c+d+e

        def NLLLoss(self, y_true, y_pred):
            y_true = tf.one_hot(y_true, CLASSES_NUM)
            y_pred = tf.math.log(tf.math.add(y_pred, 0.000001))
    #         loss = tf.einsum('mn, mn -> mn', y_true, y_pred)
            loss = (y_true*y_pred)
            loss = -(tf.reduce_sum(loss, axis=1))
            return loss

# training

In [91]:
NUM_TRAINING_IMAGES = count_data_items(TRAINING_FILENAMES)
NUM_VALIDATION_IMAGES = count_data_items(VALIDATION_FILENAMES)
NUM_TEST_IMAGES = count_data_items(TEST_FILENAMES)
STEPS_PER_EPOCH = NUM_TRAINING_IMAGES // BATCH_SIZE
VALIDATION_STEPS = (NUM_VALIDATION_IMAGES // BATCH_SIZE)-1 # The "-(-//)" trick rounds up instead of down :-)
TEST_STEPS = NUM_TEST_IMAGES // BATCH_SIZE        # The "-(-//)" trick rounds up instead of down :-)
PER_REPLICA_BATCH_SIZE = BATCH_SIZE // strategy.num_replicas_in_sync
print('Dataset: {} training images, {} validation images, {} unlabeled test images'.format(NUM_TRAINING_IMAGES, NUM_VALIDATION_IMAGES, NUM_TEST_IMAGES))
print(STEPS_PER_EPOCH, VALIDATION_STEPS)

per_replica_batch_size = BATCH_SIZE // strategy.num_replicas_in_sync
train_dataset = strategy.experimental_distribute_datasets_from_function(lambda _: get_training_dataset())
val_dataset = strategy.experimental_distribute_datasets_from_function(lambda _: get_validation_dataset())
train_iterator = iter(train_dataset)
val_iterator = iter(val_dataset)
lr_schedule = keras.optimizers.schedules.PiecewiseConstantDecay(
    boundaries=[50, 100, 150],
    values=[0.001, 0.0005, 0.0001, 0.00005]  # [0.5, 0.1, 0.01, 0.005]
)
with strategy.scope():
    model = ACnet(input_shape=(448, 448, 3), in_channel=512, classes=104, size=28,firstStage=True)
#     model.load_weights('./model.h5')
    loss_fn = myLoss(1, 1)
    optimizer = tf.optimizers.Adam(learning_rate=lr_schedule)
    training_accuracy = tf.keras.metrics.SparseCategoricalAccuracy('training_accuracy', dtype=tf.float32)
    val_accuracy =tf.keras.metrics.SparseCategoricalAccuracy('val_accuracy', dtype=tf.float32)
    training_loss = tf.keras.metrics.Mean('training_loss', dtype=tf.float32)
    
"""speed up with tf.funcion"""
@tf.function
def train_step(iterator):
    def train_fn(inputs):
        x, y = inputs
        with tf.GradientTape() as tape:
            logits = model(x, training=True)
            loss_value = loss_fn(y, logits)
            loss_value = tf.nn.compute_average_loss(loss_value, global_batch_size=BATCH_SIZE)

        grads = tape.gradient(loss_value, model.trainable_weights)
        optimizer.apply_gradients(list(zip(grads, model.trainable_weights)))
        training_accuracy.update_state(y, logits[-1])
        training_loss.update_state(loss_value * strategy.num_replicas_in_sync)
    
    strategy.run(train_fn, args=(next(iterator),))
    
@tf.function
def test_step(iterator):
    def test_fn(inputs):
        x, y = inputs
        val_logits = model(x, training=False)
        val_accuracy.update_state(y, val_logits[-1])
    
    strategy.run(test_fn, args=(next(iterator),))

Dataset: 12753 training images, 3712 validation images, 7382 unlabeled test images
99 28


第一阶段的训练

In [86]:
epochs = 220

for epoch in range(epochs):
    print('\nstart of epoch %d'%(epoch,))
    for step in range(STEPS_PER_EPOCH):
        train_step(train_iterator)    
    train_acc = training_accuracy.result()
    print('training acc over epoch: %.4f, %4f'%(float(train_acc), float(training_loss.result())))
    training_accuracy.reset_states()
    training_loss.reset_states()

    for step in range(VALIDATION_STEPS):
        test_step(val_iterator)
    val_acc = val_accuracy.result()
    print('validation acc over epoch: %.4f'%(float(val_acc),))
    val_accuracy.reset_states()


start of epoch 0
validation acc over epoch: 0.8728


In [ ]:
model.save_weights('./model.h5')

第二阶段的训练

In [ ]:
lr_schedule_2 = keras.optimizers.schedules.PiecewiseConstantDecay(
    boundaries=[50, 100, 150],
    values=[0.00001, 0.05, 0.01, 0.005]  # [0.5, 0.1, 0.01, 0.005]
)
with strategy.scope():
    model = ACnet(input_shape=(448, 448, 3), in_channel=512, classes=104, size=28,firstStage=False)
    model.load_weights('./model.h5')
    loss_fn = myLoss(1, 1)
    optimizer_2 = tf.optimizers.Adam(learning_rate=lr_schedule_2)
    training_accuracy_2 = tf.keras.metrics.SparseCategoricalAccuracy('training_accuracy_2', dtype=tf.float32)
    val_accuracy_2 =tf.keras.metrics.SparseCategoricalAccuracy('val_accuracy_2', dtype=tf.float32)
    training_loss_2 = tf.keras.metrics.Mean('training_loss_2', dtype=tf.float32)
    
"""speed up with tf.funcion"""
@tf.function
def train_step_2(iterator):
    def train_fn(inputs):
        x, y = inputs
        with tf.GradientTape() as tape:
            logits = model(x, training=True)
            loss_value = loss_fn(y, logits)
            loss_value = tf.nn.compute_average_loss(loss_value, global_batch_size=BATCH_SIZE)

        grads = tape.gradient(loss_value, model.trainable_weights)
        optimizer.apply_gradients(list(zip(grads, model.trainable_weights)))
        training_accuracy_2.update_state(y, logits[-1])
        training_loss_2.update_state(loss_value * strategy.num_replicas_in_sync)
    
    strategy.run(train_fn, args=(next(iterator),))
    
@tf.function
def test_step_2(iterator):
    def test_fn(inputs):
        x, y = inputs
        val_logits = model(x, training=False)
        val_accuracy_2.update_state(y, val_logits[-1])
    
    strategy.run(test_fn, args=(next(iterator),))
    
epochs = 150
print('stage two ......')
for epoch in range(epochs):
    print('\nstart of epoch %d'%(epoch,))
    for step in range(STEPS_PER_EPOCH):
        train_step_2(train_iterator)   
    train_acc = training_accuracy_2.result()
    print('training acc over epoch: %.4f, %4f'%(float(train_acc), float(training_loss_2.result())))
    training_accuracy_2.reset_states()
    training_loss_2.reset_states()

    for step in range(VALIDATION_STEPS):
        test_step_2(val_iterator)
    val_acc = val_accuracy_2.result()
    print('validation acc over epoch: %.4f'%(float(val_acc),))
    val_accuracy_2.reset_states()

stage two ......

start of epoch 0
training acc over epoch: 0.8780, 2.996365
validation acc over epoch: 0.7902

start of epoch 1
training acc over epoch: 0.9514, 1.559811
validation acc over epoch: 0.8627

start of epoch 2
training acc over epoch: 0.9694, 1.175860
validation acc over epoch: 0.8680

start of epoch 3
training acc over epoch: 0.9793, 0.955817
validation acc over epoch: 0.8809

start of epoch 4


In [ ]:
model.save_weights('./model_.h5')

In [ ]:
# with strategy.scope():
#     #img_adjust_layer = tf.keras.layers.Lambda(lambda data: tf.keras.applications.xception.preprocess_input(tf.cast(data, tf.float32)), input_shape=[*IMAGE_SIZE, 3])
#     #pretrained_model = tf.keras.applications.Xception(weights='imagenet', include_top=False)
    
#     img_adjust_layer = tf.keras.layers.Lambda(lambda data: tf.keras.applications.vgg16.preprocess_input(tf.cast(data, tf.float32)), input_shape=[*IMAGE_SIZE, 3])
#     pretrained_model = tf.keras.applications.VGG16(weights='imagenet', include_top=False)
    
#     pretrained_model.trainable = False # False = transfer learning, True = fine-tuning
    
#     model = tf.keras.Sequential([
#         img_adjust_layer,
#         pretrained_model,
#         tf.keras.layers.GlobalAveragePooling2D(),
#         tf.keras.layers.Dense(len(CLASSES), activation='softmax')
#     ])
        
# model.compile(
#     optimizer='adam',
#     loss = 'sparse_categorical_crossentropy',
#     metrics=['sparse_categorical_accuracy'],
#     # NEW on TPU in TensorFlow 24: sending multiple batches to the TPU at once saves communications
#     # overheads and allows the XLA compiler to unroll the loop on TPU and optimize hardware utilization.
#     steps_per_execution=16
# )
# model.summary()

# Training

In [ ]:
# history = model.fit(get_training_dataset(), steps_per_epoch=STEPS_PER_EPOCH, epochs=EPOCHS,
#                     validation_data=get_validation_dataset(), validation_steps=VALIDATION_STEPS)

In [ ]:
# display_training_curves(history.history['loss'], history.history['val_loss'], 'loss', 211)
# display_training_curves(history.history['sparse_categorical_accuracy'], history.history['val_sparse_categorical_accuracy'], 'accuracy', 212)

# Confusion matrix

In [ ]:
# cmdataset = get_validation_dataset(ordered=True) # since we are splitting the dataset and iterating separately on images and labels, order matters.
# images_ds = cmdataset.map(lambda image, label: image)
# labels_ds = cmdataset.map(lambda image, label: label).unbatch()
# cm_correct_labels = next(iter(labels_ds.batch(NUM_VALIDATION_IMAGES))).numpy() # get everything as one batch
# cm_probabilities = model.predict(images_ds, steps=VALIDATION_STEPS)
# cm_predictions = np.argmax(cm_probabilities, axis=-1)
# print("Correct   labels: ", cm_correct_labels.shape, cm_correct_labels)
# print("Predicted labels: ", cm_predictions.shape, cm_predictions)

In [ ]:
# cmat = confusion_matrix(cm_correct_labels, cm_predictions, labels=range(len(CLASSES)))
# score = f1_score(cm_correct_labels, cm_predictions, labels=range(len(CLASSES)), average='macro')
# precision = precision_score(cm_correct_labels, cm_predictions, labels=range(len(CLASSES)), average='macro')
# recall = recall_score(cm_correct_labels, cm_predictions, labels=range(len(CLASSES)), average='macro')
# cmat = (cmat.T / cmat.sum(axis=1)).T # normalized
# display_confusion_matrix(cmat, score, precision, recall)
# print('f1 score: {:.3f}, precision: {:.3f}, recall: {:.3f}'.format(score, precision, recall))

# Predictions

In [ ]:
# test_ds = get_test_dataset(ordered=True) # since we are splitting the dataset and iterating separately on images and ids, order matters.

# print('Computing predictions...')
# test_images_ds = test_ds.map(lambda image, idnum: image)
# probabilities = model.predict(test_images_ds, steps=TEST_STEPS)
# predictions = np.argmax(probabilities, axis=-1)
# print(predictions)

# print('Generating submission.csv file...')
# test_ids_ds = test_ds.map(lambda image, idnum: idnum).unbatch()
# test_ids = next(iter(test_ids_ds.batch(NUM_TEST_IMAGES))).numpy().astype('U') # all in one batch
# np.savetxt('submission.csv', np.rec.fromarrays([test_ids, predictions]), fmt=['%s', '%d'], delimiter=',', header='id,label', comments='')
# !head submission.csv

# Visual validation

In [ ]:
# dataset = get_validation_dataset()
# dataset = dataset.unbatch().batch(20)
# batch = iter(dataset)

In [ ]:
# # run this cell again for next set of images
# images, labels = next(batch)
# probabilities = model.predict(tf.cast(images, tf.float32))
# predictions = np.argmax(probabilities, axis=-1)
# display_batch_of_images((images, labels), predictions)